# <div align="center"> Experiments

In [1]:
from utils import *
from net_bml import Net
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.special import logsumexp

# 1. Choose a dataset

Choose the data between the following dataset:

Regression problems:

- `bostonHousing`
- `concrete`
- `energy`
- `kin8mn`
- `naval-propulsion-plant`
- `power-plant`
- `protein-tertiary-sctructure`
- `wine-quality-red`
- `yatch`

Classification problems:

- `MNIST`

Then select the batch sizes (`batch_size_train`, `batch_size_test`) used for the next training

In [2]:
data_name = "bostonHousing"
batch_size_train= 128
batch_size_test= 50

training_set, testing_set, train_loader, test_loader, x_mean, x_std, y_mean, y_std = load_data(data_name=data_name,
                                                                                                          batch_size_train=batch_size_train, 
                                                                                                          batch_size_test = batch_size_test,
                                                                                                          normalize=True)

print("The shape of the training set is:")
print(training_set.data.size())
print("The shape of the testing set is:")
print(testing_set.data.size())

The shape of the training set is:
torch.Size([455, 13])
The shape of the testing set is:
torch.Size([51, 13])


# III. Create the network

First let's choose a network architecture. We only study here linear layers (ReLu activation) separated by a dropout layer. The weights $\W_{i}$ and bias $b_{i}$ of the linear layers are regularized ($L_2$) with the regularization parameter $\lambda$.

You have to set the following parameters:
- the number of input data $N$ (eg.`N=1000`)
- the number of input features $n_{in}$ (eg. `n_in=784`)
- the number of classes $n_{out}$ (eg. `n_out=10`)
- number of layers and neurons (eg. `layer_sizes=[256,256,256]`)
- the dropout rate $p$ (eg. `dropout_rate=0.5`)
- $\tau$ (eg. `tau=0.5`)

Please note the regularization parameter $\lambda$ is compute according to the following formula:


<div align="center"> $\lambda=\frac{pl^{2}}{2N\tau}$
    
where $l$ is the prior-length scale. It is arbitrary set of `1e-2`

In [5]:
# Set the parameters
N=training_set.data.size()[0]
n_in=training_set.data.size()[1]
n_out=1
layer_sizes=[100,]
dropout_rate= 0.005 
tau=0.1
l=1e-2
lbda = l**2 * (1 - dropout_rate) / (2. * N * tau)

# Create the network
network = Net(N=N,
              n_in=n_in,
              n_out=n_out,
              layer_sizes=layer_sizes,
              dropout_rate=dropout_rate,
              tau=tau).float()

print(network)

Net(
  (model): Sequential(
    (Dropout0): Dropout(p=0.005, inplace=False)
    (Linear0): Linear(in_features=13, out_features=100, bias=True)
    (Relu): ReLU()
    (Dropout1): Dropout(p=0.005, inplace=False)
    (Linear1): Linear(in_features=100, out_features=1, bias=True)
  )
)


# III - Train the model

Here are the useful functions to run the training.

You have to choose the type of the problem, either classification or regression:
- `problem="classification"` or `problem=regression`

It will define the loss to use, either NLL for classification or RMSE for regression.

Then, you have to choose the parameters of the training:
- number of epochs (eg. `n_epochs=10`)
- learning rate (eg. `learning_rate=0.01`)
- momentum (eg. `momentum=0.5`)

In [6]:
# Type of the problem
problem="regression"

# Training parameters
n_epochs = 300
learning_rate = 1e-8
momentum = 0.5

optimizer = optim.Adam(network.parameters(), 
                      lr=learning_rate,
                     weight_decay=lbda)

In [7]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

def train(network,epoch,aff=False):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        
        optimizer.zero_grad()
        

        if data_name=="MNIST":
            data=data.squeeze(dim=1)
            data=data.flatten(1,2)
            
        output = network(data)
        
        if problem=="classification":
            loss = F.nll_loss(output, target)
        if problem=="regression":
            loss = F.mse_loss(output,target)
            
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            if aff:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

def test(network,aff=False):
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            
            if data_name=="MNIST":
                data=data.squeeze(dim=1)
                data=data.flatten(1,2)
                
            output = network(data)
            
            if problem=="classification":
                test_loss += F.nll_loss(output, target, size_average=False).item()
                pred = output.data.max(1, keepdim=True)[1]
                correct += pred.eq(target.data.view_as(pred)).sum()
            if problem=="regression":
                test_loss += F.mse_loss(output,target,reduction="mean").item()
                
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    if aff:
        if problem=="classification":
            print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                test_loss, correct, len(test_loader.dataset),
                100. * correct / len(test_loader.dataset)))
        if problem=="regression":
            print('\nTest set: Avg. loss: {:.4f}\n'.format(test_loss))


Then let's run the training:

In [8]:
for epoch in range(1, n_epochs + 1):
    train(network,epoch)
    test(network)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Using a target size (torch.Size([71])) that is different to the input size (torch.Size([71, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torc

# IV - Perform the prediction

In [9]:
def prediction(network,testing_set,normalize=True):

    network.eval()
    with torch.no_grad():
        standard_pred = network(testing_set.data)

    rmse_standard_pred = np.mean((testing_set.targets.data.numpy() - standard_pred.squeeze(dim=1).data.numpy())**2.)**0.5
    if normalize:
        rmse_standard_pred*=y_std
        rmse_standard_pred+=y_mean
    
    T = 10000

    Yt_hat=[]
    for _ in range(T):
        Yt_hat.append(network(testing_set.data).data.numpy())
        optimizer.step()
    Yt_hat=np.array(Yt_hat)
    if normalize:
        Yt_hat = Yt_hat * y_std + y_mean
    MC_pred = np.mean(Yt_hat, 0)
    rmse = np.mean((testing_set.data.numpy() - MC_pred)**2.)**0.5

    # We compute the test log-likelihood
    ll = (logsumexp(-0.5 * tau * (testing_set.data.numpy() - Yt_hat)**2., 0) - np.log(T)
        - 0.5*np.log(2*np.pi) + 0.5*np.log(tau))
    test_ll = np.mean(ll)

    # We are done!
    print(rmse_standard_pred)
    print(rmse)
    print(test_ll)
    
    return rmse_standard_pred, rmse, test_ll

prediction(network,testing_set,normalize=True)

30.33067841214774
21.833019147558502
-25.904394


(30.33067841214774, 21.833019147558502, -25.904394)

# VI - Grid search

In [10]:
for dropout_rate in [0.005, 0.01, 0.05, 0.1]:
    for tau in [0.1, 0.15, 0.2]:
        print("tau = {} ; dropout_rate = {}".format(dropout_rate,tau))
        # Set the parameters
        N=training_set.data.size()[0]
        n_in=training_set.data.size()[1]
        n_out=1
        layer_sizes=[100,]
        l=1e-2
        lbda = (dropout_rate*l**2)/(2*N*tau)

        # Create the network
        network = Net(N=N,
                      n_in=n_in,
                      n_out=n_out,
                      layer_sizes=[256],
                      dropout_rate=0.5,
                      tau=0.5).float()

        # Type of the problem
        problem="regression"

        # Training parameters
        n_epochs = 100
        learning_rate = 1e-8
        momentum = 0.5

        optimizer = optim.Adam(network.parameters(), 
                              lr=learning_rate,
                             weight_decay=lbda)

        train_losses = []
        train_counter = []
        test_losses = []
        test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]
        
        print("Training")
        for epoch in range(1, n_epochs + 1):
            train(network,epoch)
            test(network)
    
        prediction(network, testing_set)

tau = 0.005 ; dropout_rate = 0.1
Training


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Using a target size (torch.Size([71])) that is different to the input size (torch.Size([71, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torc

30.452843786471128
23.190640054401527
-28.96031
tau = 0.005 ; dropout_rate = 0.15
Training
29.344012555421838
21.684931094478156
-37.133213
tau = 0.005 ; dropout_rate = 0.2
Training
30.448301180775772
21.92618592903315
-49.797543
tau = 0.01 ; dropout_rate = 0.1
Training
31.545179217439365
23.93256058291063
-30.708101
tau = 0.01 ; dropout_rate = 0.15
Training
29.400744839142693
21.804371890630602
-37.523056
tau = 0.01 ; dropout_rate = 0.2
Training
31.545204062692633
22.915098638353463
-54.229984
tau = 0.05 ; dropout_rate = 0.1
Training
31.248834670758256
25.625427540259572
-34.902348
tau = 0.05 ; dropout_rate = 0.15
Training
30.1684748879866
24.09820808301937
-45.419292
tau = 0.05 ; dropout_rate = 0.2
Training
30.60469858194638
21.031942869112434
-45.95659
tau = 0.1 ; dropout_rate = 0.1
Training
30.21254818029489
22.226585841376146
-26.770727
tau = 0.1 ; dropout_rate = 0.15
Training
28.744727977954348
21.632440775291272
-36.963036
tau = 0.1 ; dropout_rate = 0.2
Training
30.8747464606475

# V - For MNIST

In [ ]:
#result=np.zeros((1,10))

#network.train()

#for i in range(100):
#    output=network(training_set.data[0].float())
#    if problem=="classification":
#        loss = F.nll_loss(output.unsqueeze(0), training_set.targets[0].unsqueeze(dim=0))
#    loss.backward()
#    pred = output.argmax()
#    print(pred,training_set.targets[0])
#    result=np.concatenate((result,[output.data.numpy()]),axis=0)
    
#result=result[1:]